In [1]:
%%capture
!pip install nltk transformers
!pip install sentencepiece

In [2]:
import nltk
from nltk.corpus import brown
from transformers import TFAlbertModel, AlbertTokenizer, AlbertConfig
import numpy as np
import csv
from nltk.tokenize import word_tokenize
import os
from tqdm import tqdm

import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
nltk.download('brown')
nltk.download('punkt')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
model_name = "albert-base-v2"

original_config = AlbertConfig.from_pretrained(model_name)

In [6]:
# Create a new configuration with reduced hidden size
reduced_hidden_size = 12  # Replace with your desired hidden size
reduced_num_hidden_layers = 1  # Choose a smaller number
reduced_num_attention_heads = 2  # Choose a smaller number
new_config = AlbertConfig(
    vocab_size=original_config.vocab_size,
    hidden_size=reduced_hidden_size,
    num_hidden_layers=reduced_num_hidden_layers,
    num_attention_heads=reduced_num_attention_heads,
)

In [7]:
model = TFAlbertModel.from_pretrained(model_name, config=new_config, ignore_mismatched_sizes=True)
tokenizer = AlbertTokenizer.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['albert.encoder.embedding_hidden_mapping_in.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn.bias', 'albert.encoder.embedding_hidden_mapping_in.weight', 'predictions.LayerNorm.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.LayerNorm.weight', 'predictions.LayerNorm.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight', 'predictions.decoder.bias', 'albert.pooler.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn_output.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.dense

In [8]:
def embed_words(words):
    tokens = tokenizer.batch_encode_plus(words, padding='max_length', truncation=True, return_tensors='tf', max_length=64)
    outputs = model(tokens['input_ids'])
    embeddings = outputs.last_hidden_state
    averaged_embeddings = tf.reduce_mean(embeddings, axis=1)
    return averaged_embeddings

In [9]:
dataset_path = r"/content/drive/MyDrive/Reuters-21578/reuters/reuters/reuters/training"
dataset_dirs = os.listdir(dataset_path)

In [10]:
data = []
for i in tqdm(dataset_dirs):
    with open(f"{dataset_path}/{i}", 'r') as f:
        content = f.read()
        data.append(content)

100%|██████████| 7769/7769 [00:30<00:00, 256.92it/s]


In [11]:
words = []
for text in data:
  temp_words = word_tokenize(text)
  words.append(temp_words)

In [12]:
from itertools import chain

words_flattened = list(chain(*words))


In [13]:
len(words_flattened)

1135633

In [14]:
words_flattened = [str(element) for element in words_flattened]

In [ ]:
embeddings = embed_words(words_flattened)

In [ ]:
# Save embeddings and metadata in TSV format
tsv_file = 'word_embeddings.tsv'
metadata_file = 'metadata.tsv'

# Save embeddings
np.savetxt(tsv_file, embeddings, delimiter='\t')

# Save metadata
with open(metadata_file, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerows([[word] for word in words_flattened])

In [ ]:
len(embeddings)